# A/B Testing

## Background

Data scientists sometimes need to do experiments about user interface in web design and product marketing. A/B testing is a common approach to test the hypothesis. For example, whether a new interface B is better than the existing interface A.

**Control group** is a standard, existing, without-new-things group.

**Treatment group (Test group)** is a group with a new feature or a specific thing we want to test. 

A/B testing typically measures, for example

- Number of clicks
- Number of purchases
- Duration of seeing web apges

## Hypothesis test

Something that A/B testing usually establishes. When we see difference between A and B, the hypothesis test will tell us *whether it's due to random chance or true difference*. 

A/B testing typically uses **one-way** hypothesis test (**one-tail** hypothesis test). One-way hypothesis test treats one group as default and test whether the other group is better.

**Z-test** and **t-test** are hypothesis test. Both are the *same* in the following points.

- Decide if the **mean** from the sample is different from a certain value (or more than a value, or less than a value)
- Assume the samples follow a **normal distribution**.
- Use the same test statistic below. $s$ is the standard deviation. $n$ is the sample size

$$
\frac{\hat{\mu} - \mu_0}{\frac{s}{\sqrt{n}}}
$$

But they are *different* in the following.

Z-test uses **standard normal distribution**.

t-test uses **t-distribution**. It has **degrees of freedom**. As it gets large, t-distribution gets closer to standard normal distribution. When the sample size is small, t-test is more appropriate than Z-test.

### Type I error

### Type II error

## Random permutation test

Randomization test, resampling permutation procedure.

When samples don't follow a normal distribution

## Multiple A/B tests

Yes, we can. But we cannot purely run the A/B tests multiple times. We need to consider the following.

### Bonferroni correction (adjustment)

Divide significance level $\alpha$ by the number of multiple test $n$, so $\frac{\alpha}{n}$

### F-test

xxx

### Multi-arm bandit

xxx

## Compute sample size

We can compute sample size if we have **effect size**, **power**, and **significance level**.

The **effect size** is the amount of change we hope to have with a new feature. For example, with a new feature, if we want the click rate to increase by 10%, the 10% is the effect size.

The **power** is the probability of detecting the effect size in A/B testing.

The **significance level** is the probability of incorrectly saying we have the effect when the effect actually doesn't exist.

## Experimental design

### Multiple features

Button color and button location

### Interpret A/B testing result

new UI effect

### Apply A/B testing to price

### Randomization

Random bucketing

### Non-normal distribution

Mann-whitney, etc.

## Terminology

- Churn rate
  - The rate at which a customer stops doing business with a company, or stops subscribing to a service.
- Researcher bias
  - Selecting a test statistic after an experiment, but the test statistic needs to be established beforehand. 

## Reference

- O'REILLY Practical Statistics for Data Scientist, 50+ Essential Concepts Using R and Python
- [How to A/B Test Your Pricing (And Why It Might Be a Bad Idea)](https://blog.hubspot.com/marketing/price-testing)
- [Common Mistakes During A/B Testing](https://towardsdatascience.com/common-mistakes-during-a-b-testing-bdb9eefdc7f0)
- [Type I & Type II Errors | Differences, Examples, Visualizations](https://www.scribbr.com/statistics/type-i-and-type-ii-errors/)